In [1]:
%matplotlib inline
import numpy as np
from tqdm import tqdm, tqdm_notebook
from os import makedirs
from os.path import exists, join
from shutil import copyfile
from skimage import io, transform

In [2]:
data_dir = "../LPDwithoutseparation/"

In [3]:
if not exists(data_dir):
    makedirs(data_dir)

In [4]:
x_train_paths = np.load("../Cache/LPD/original/x_train.npy")
x_test_paths = np.load("../Cache/LPD/original/x_test.npy")
x_validation_paths = np.load("../Cache/LPD/original/x_validation.npy")
y_train_paths = np.load("../Cache/LPD/original/y_train.npy")
y_test_paths = np.load("../Cache/LPD/original/y_test.npy")
y_validation_paths = np.load("../Cache/LPD/original/y_validation.npy")

In [6]:
print(x_train_paths[0], y_train_paths[0])
print(x_train_paths[0].split("/")[-1][:-4])
print(join(data_dir, x_train_paths[0].split("/")[-1][:-4]) + ".jpg")
im = io.imread(x_train_paths[0])
print(im.shape)
print(type(y_train_paths[0][0]))

../UFPR-ALPR-dataset/training/track0008/track0008[18].png [422 862  25  77]
track0008[18]
../LPDwithoutseparation/track0008[18].jpg
(1080, 1920, 3)
<class 'numpy.int64'>


In [7]:
print(x_train_paths.shape, y_train_paths.shape)
X = np.concatenate((x_train_paths, x_test_paths, x_validation_paths))
Y = np.concatenate((y_train_paths, y_test_paths, y_validation_paths))
print(X.shape, Y.shape)
print(x_test_paths[0], X[1800])

(1800,) (1800, 4)
(4500,) (4500, 4)
../UFPR-ALPR-dataset/testing/track0109/track0109[10].png ../UFPR-ALPR-dataset/testing/track0109/track0109[10].png


# Spliting

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.125)

In [10]:
print(y_train.shape, y_test.shape, y_validation.shape)

(3150, 4) (900, 4) (450, 4)


In [11]:
np.save("../Cache/LPD/new/x_train.npy", x_train)
np.save("../Cache/LPD/new/x_test.npy", x_test)
np.save("../Cache/LPD/new/x_validation.npy", x_validation)
np.save("../Cache/LPD/new/y_train.npy", y_train)
np.save("../Cache/LPD/new/y_test.npy", y_test)
np.save("../Cache/LPD/new/y_validation.npy", y_validation)

In [12]:
dim_x = 416
dim_y = 416

# Train

In [13]:
train_dir = "../LPD/training/"
if not exists(train_dir):
    makedirs(train_dir)
xy = np.column_stack((x_train, y_train))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    bb = np.asarray(path[1:], dtype=int)
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(train_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write(" ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=3150), HTML(value='')))

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


# Test

In [14]:
test_dir = "../LPD/testing/"
if not exists(test_dir):
    makedirs(test_dir)
xy = np.column_stack((x_test, y_test))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    bb = np.asarray(path[1:], dtype=int)
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(train_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write(" ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=900), HTML(value='')))

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


# Validation

In [15]:
validation_dir = "../LPD/validation/"
if not exists(validation_dir):
    makedirs(validation_dir)
xy = np.column_stack((x_validation, y_validation))
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    bb = np.asarray(path[1:], dtype=int)
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    with open(join(train_dir, path[0].split("/")[-1][:-4]) + ".txt", "w") as file:
        file.write(" ".join([str(a) for a in bb]) + '\n')
    io.imsave(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)

HBox(children=(IntProgress(value=0, max=450), HTML(value='')))

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
